In [1]:
import depthai as dai
import pyrealsense2 as rs
import numpy as np
import cv2

In [2]:
window_size = (640, 360)

In [3]:
pipeline_rs = rs.pipeline()
config_rs = rs.config()
config_rs.enable_stream(rs.stream.color, window_size[0], window_size[1], rs.format.bgr8, 30)
config_rs.enable_stream(rs.stream.depth, window_size[0], window_size[1], rs.format.z16, 30)

# Start streaming
pipeline_rs.start(config_rs)
rsalign = rs.align(rs.stream.color)

In [4]:
while True:
    frames = pipeline_rs.wait_for_frames()
    frames = rsalign.process(frames)
    color_frame_rs = frames.get_color_frame()
    depth_frame_rs = frames.get_depth_frame()

    frame_rs = np.asanyarray(color_frame_rs.get_data())
    depth_rs = np.asanyarray(depth_frame_rs.get_data(), dtype=np.float32)
    
    cv2.imshow("RS", frame_rs)
    
    if cv2.waitKey(10) & 0xFF == ord("q"):
        pipeline_rs.stop()
        cv2.destroyAllWindows()
        break

In [5]:
pipeline_oak = dai.Pipeline()

cam_rgb = pipeline_oak.create(dai.node.ColorCamera)
cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setFps(30)

mono_left = pipeline_oak.create(dai.node.MonoCamera)
mono_right = pipeline_oak.create(dai.node.MonoCamera)
stereo = pipeline_oak.create(dai.node.StereoDepth)

stereo.setDepthAlign(dai.CameraBoardSocket.RGB)
stereo.setOutputSize(window_size[0], window_size[1])

mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
mono_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
mono_right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)

stereo.setDefaultProfilePreset(dai.node.StereoDepth.PresetMode.HIGH_DENSITY)
mono_left.out.link(stereo.left)
mono_right.out.link(stereo.right)

xout_rgb = pipeline_oak.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
cam_rgb.video.link(xout_rgb.input)

xout_depth = pipeline_oak.create(dai.node.XLinkOut)
xout_depth.setStreamName("depth")
stereo.depth.link(xout_depth.input)

/tmp/ipykernel_29615/4019615464.py:4: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
/tmp/ipykernel_29615/4019615464.py:12: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  stereo.setDepthAlign(dai.CameraBoardSocket.RGB)
/tmp/ipykernel_29615/4019615464.py:15: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  mono_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
/tmp/ipykernel_29615/4019615464.py:16: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  mono_right.setBoardSocket(dai.CameraBoardSocket.RIGHT)


In [6]:
device_oak = dai.Device(pipeline_oak)
rgb_queue_oak = device_oak.getOutputQueue(name="rgb", maxSize=2, blocking=False)
depth_queue_oak = device_oak.getOutputQueue(name="depth", maxSize=2, blocking=False)

while True:
    rgb_frame_oak = rgb_queue_oak.get()
    depth_frame_oak = depth_queue_oak.get()

    frame_oak = rgb_frame_oak.getCvFrame()
    depth_oak = depth_frame_oak.getFrame()
    depth_oak = depth_oak.astype(np.float32)

    frame_oak = cv2.resize(frame_oak, window_size)
    cv2.imshow("OAK", frame_oak)
    
    if cv2.waitKey(10) & 0xFF == ord("q"):
        cv2.destroyAllWindows()
        break